# TORCH.AUYOGRAD를 사용한 자동 미분
- 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파이다. 이 때 매개변수느 주어진 매개변수에 대한 송싱함수의 gradient로 조정된다.
- pytorch에 torch.autograd()는 자동 미분 엔진이다.

### Tensor, fuction과 연산그래프(Computational graph)
- 신경망에서 w와b는 최적화를 해야 하는 매개변수이다. 따라서 이러한 변수들에 대한 손실 함수의 변화도르 계산할 수 있어야한다.
- 이를 위해서 해당 텐서에 requires_grad속성을 설정한다.

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x104aef490>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x104aef370>


### 변화도(Gradient) 계산하기
- 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대란 손실함수의 도함수(dericative)를 계산해야한다.
- 이러한 도함수를 계산하기 위해 loss.backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져온다.

In [3]:
# requires_grad가 True인 노드들의 grad만 구할 수 있다. backward호출이 여러 번 필요하면 retrain_graph=True를 사용
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2508, 0.2877, 0.2710],
        [0.2508, 0.2877, 0.2710],
        [0.2508, 0.2877, 0.2710],
        [0.2508, 0.2877, 0.2710],
        [0.2508, 0.2877, 0.2710]])
tensor([0.2508, 0.2877, 0.2710])


### 변화도 추적 멈추기
- requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원한다.
- 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파연산만 필요한 경우헤는 torch.no_grad()블록으로 둘러싸서 연산 추적을 멈출 수 있다. 

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

# 동일한 결과 dstach()사용
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

True
False
False


변화도 투적을 멈춰야 하는 이유
- 신경망의 일부 매개변수를 고정된 매개변수로 표기한다. 이는 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오이다.
- 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단게만 수행할 때 연산 속도가 향상된다.

### 연산 그래프에 대한 추가 정보
- autograd는 텐서의 실행된 모든 연산들의 기록을 Function객체로 구성된 비순화 그래프(DAG)에 저장한다.
- 이 DAG의 leave는 입력 텐서이고, root는 결과 텐서이다.
- 따라서 root에서 leave로 추적하면 연쇄 법칙에 따라 gradient를 자동으로 계산한다.

순전파 단계에서, autograd는 다음 두 가지 작업을 동시에 수행합니다:
- 요청된 연산을 수행하여 결과 텐서를 계산하고,
- DAG에 연산의 변화도 기능(gradient function) 를 유지(maintain)합니다.      
      
역전파 단계는 DAG 뿌리(root)에서 .backward() 가 호출될 때 시작됩니다. autograd는 이 때:
- 각 .grad_fn 으로부터 변화도를 계산하고,
- 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate),
- 연쇄 법칙을 사용하여, 모든 잎(leaf) 텐서들까지 전파(propagate)합니다.

### 선택적으로 읽기(Optional Reading): 텐서 변화도와 야코비안 곱 (Jacobian Product)
- pytorch에서는 실제 gradient가 아닌 야코비안 곱(jacobian product)을 계산한다.
- x에 대한 y의 변화도는 야코비안 행렬로 주어진다.
---
무슨 말인지 모르겠지만 일반적으로 backward로 계산하는 것보다 빠르다는 것 같음..아마도...

In [6]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
